# 实现决策树

实验内容：  
使用LendingClub Safe Loans数据集：
1. 实现信息增益、信息增益率、基尼指数三种划分标准
2. 使用给定的训练集完成三种决策树的训练过程
3. 计算三种决策树在最大深度为6时测试集上的精度，查准率，查全率，F1值

在这部分，我们会实现一个很简单的二叉决策树

## 1. 读取数据

In [1]:
# 导入类库
import pandas as pd
import numpy as np
import json

In [2]:
# 导入数据
loans = pd.read_csv('data/lendingclub/lending-club-data.csv', low_memory=False)

数据中有两列是我们想预测的指标，一项是safe_loans，一项是bad_loans，分别表示正例和负例，我们对其进行处理，将正例的safe_loans设为1，负例设为-1，删除bad_loans这列

In [3]:
# 对数据进行预处理，将safe_loans作为标记
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
del loans['bad_loans']

我们只使用grade, term, home_ownership, emp_length这四列作为特征，safe_loans作为标记，只保留loans中的这五列

In [4]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

查看前五行数据

In [5]:
loans.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


## 2. 划分训练集和测试集

In [6]:
from sklearn.utils import shuffle
loans = shuffle(loans, random_state = 34)

split_line = int(len(loans) * 0.6)
train_data = loans.iloc[: split_line]
test_data = loans.iloc[split_line:]

## 3. 特征预处理

可以看到所有的特征都是离散类型的特征，需要对数据进行预处理，使用one-hot编码对其进行处理。

one-hot编码的思想就是将离散特征变成向量，假设特征$A$有三种取值$\{a, b, c\}$，这三种取值等价，如果我们使用1,2,3三个数字表示这三种取值，那么在计算时就会产生偏差，有一些涉及距离度量的算法会认为，2和1离得近，3和1离得远，但这三个值应该是等价的，这种表示方法会造成模型在判断上出现偏差。解决方案就是使用一个三维向量表示他们，用$[1, 0, 0]$表示a，$[0, 1, 0]$表示b，$[0, 0, 1]$表示c，这样三个向量之间的距离就都是相等的了，任意两个向量在欧式空间的距离都是$\sqrt{2}$。这就是one-hot编码是思想。

pandas中使用get_dummies生成one-hot向量

In [7]:
def one_hot_encoding(data, features_categorical):
    '''
    Parameter
    ----------
    data: pd.DataFrame
    
    features_categorical: list(str)
    '''
    
    # 对所有的离散特征遍历
    for cat in features_categorical:
        
        # 对这列进行one-hot编码，前缀为这个变量名
        one_encoding = pd.get_dummies(data[cat], prefix = cat)
        
        # 将生成的one-hot编码与之前的dataframe拼接起来
        data = pd.concat([data, one_encoding],axis=1)
        
        # 删除掉原始的这列离散特征
        del data[cat]
    
    return data

首先对训练集生成one-hot向量，然后对测试集生成one-hot向量，这里需要注意的是，如果训练集中，特征$A$的取值为$\{a, b, c\}$，这样我们生成的特征就有三列，分别为$A\_a$, $A\_b$, $A\_c$，然后我们使用这个训练集训练模型，模型就就会考虑这三个特征，在测试集中如果有一个样本的特征$A$的值为$d$，那它的$A\_a$，$A\_b$，$A\_c$就都为0，我们不去考虑$A\_d$，因为这个特征在训练模型的时候是不存在的。

In [8]:
train_data = one_hot_encoding(train_data, features)

In [9]:
train_data.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
84320,1,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
121308,-1,True,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
58376,-1,False,False,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
66430,1,True,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
65344,1,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


获取所有特征的名字

In [10]:
one_hot_features = train_data.columns.tolist()
one_hot_features.remove(target)
one_hot_features

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year']

接下来是对测试集进行one_hot编码，但只要保留出现在one_hot_features中的特征即可·

In [11]:
test_data_tmp = one_hot_encoding(test_data, features)

In [12]:
# 创建一个空的DataFrame
test_data = pd.DataFrame(columns = train_data.columns)
for feature in train_data.columns:
    # 如果训练集中当前特征在test_data_tmp中出现了，将其复制到test_data中
    if feature in test_data_tmp.columns:
        test_data[feature] = test_data_tmp[feature].copy()
    else:
        # 否则就用全为0的列去替代
        test_data[feature] = np.zeros(test_data_tmp.shape[0], dtype = 'uint8')

In [13]:
test_data.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
37225,1,False,False,False,False,True,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
101585,-1,False,True,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
31865,1,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
97692,1,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,False
88181,1,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False


In [14]:
train_data.shape

(73564, 25)

In [15]:
test_data.shape

(49043, 25)

**处理完后，所有的特征都是0和1，标记是1和-1**，以上就是数据预处理流程

## 4. 实现3种特征划分准则

决策树中有很多常用的特征划分方法，比如信息增益、信息增益率、基尼指数

我们需要实现一个函数，它的作用是，给定决策树的某个结点内的所有样本的标记，让它计算出对应划分指标的值是多少

接下来我们会实现上述三种划分指标

**这里我们约定，将所有特征取值为0的样本，划分到左子树，特征取值为1的样本，划分到右子树**

### 4.1 信息增益
信息熵：
$$
\mathrm{Ent}(D) = - \sum^{\vert \mathcal{Y} \vert}_{k = 1} p_k \mathrm{log}_2 p_k
$$

信息增益：
$$
\mathrm{Gain}(D, a) = \mathrm{Ent}(D) - \sum^{V}_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Ent}(D^v)
$$

计算信息熵时约定：若$p = 0$，则$p \log_2p = 0$

**下面的函数需要填写两个部分**

In [21]:
def information_entropy(labels_in_node):
    '''
    求当前结点的信息熵
    
    Parameter
    ----------
    labels_in_node: np.ndarray, 如[-1, 1, -1, 1, 1]
    
    Returns
    ----------
    float: information entropy
    '''
    # 统计样本总个数
    num_of_samples = labels_in_node.shape[0]
    
    if num_of_samples == 0:
        return 0
    
    # 统计出标记为1的个数
    num_of_positive = len(labels_in_node[labels_in_node == 1])
    
    # 统计出标记为-1的个数
    num_of_negative = len(labels_in_node[labels_in_node == -1])             # YOUR CODE HERE
    
    # 统计正例的概率
    prob_positive = num_of_positive / num_of_samples
    
    # 统计负例的概率
    prob_negative = num_of_negative / num_of_samples               # YOUR CODE HERE
    
    if prob_positive == 0:
        positive_part = 0
    else:
        positive_part = prob_positive * np.log2(prob_positive)
    
    if prob_negative == 0:
        negative_part = 0
    else:
        negative_part = prob_negative * np.log2(prob_negative)
    
    return - ( positive_part + negative_part )

下面是6个测试样例

In [22]:
# 信息熵测试样例1
example_labels = np.array([-1, -1, 1, 1, 1])
print(information_entropy(example_labels)) # 0.97095

# 信息熵测试样例2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
print(information_entropy(example_labels)) # 0.86312
    
# 信息熵测试样例3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
print(information_entropy(example_labels)) # 0.86312

# 信息熵测试样例4
example_labels = np.array([-1] * 9 + [1] * 8)
print(information_entropy(example_labels)) # 0.99750

# 信息熵测试样例5
example_labels = np.array([1] * 8)
print(information_entropy(example_labels)) # 0

# 信息熵测试样例6
example_labels = np.array([])
print(information_entropy(example_labels)) # 0

0.9709505944546686
0.863120568566631
0.863120568566631
0.9975025463691153
-0.0
0


接下来完成计算所有特征的信息增益的函数  
**需要填写三个部分**

In [23]:
def compute_information_gains(data, features, target, annotate = False):
    '''
    计算所有特征的信息增益
    
    Parameter
    ----------
        data: pd.DataFrame，传入的样本，带有特征和标记的dataframe
        
        features: list(str)，特征名组成的list
        
        target: str, 标记(label)的名字
        
        annotate, boolean，是否打印所有特征的信息增益值，默认为False
        
    Returns
    ----------
        information_gains: dict, key: str, 特征名
                                 value: float，信息增益
    '''
    
    # 我们将每个特征划分的信息增益值存储在一个dict中
    # 键是特征名，值是信息增益值
    information_gains = dict()
    
    # 对所有的特征进行遍历，使用信息增益对每个特征进行计算
    for feature in features:
        
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的信息熵
        left_entropy = information_entropy(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target) / (len(left_split_target) + len(right_split_target))

        # 计算右子树的信息熵
        right_entropy = information_entropy(right_split_target)                         # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight = len(right_split_target) / (len(left_split_target) + len(right_split_target))                  
        
        # 计算当前结点的信息熵
        current_entropy = information_entropy(data[target])
            
        # 计算使用当前特征划分的信息增益
        gain = current_entropy - (left_weight * left_entropy + right_weight * right_entropy)                 # YOUR CODE HERE
        
        # 将特征名与增益值以键值对的形式存储在information_gains中
        information_gains[feature] = gain
        
        if annotate:
            print(" ", feature, gain)
            
    return information_gains

In [24]:
# 信息增益测试样例1
print(compute_information_gains(train_data, one_hot_features, target)['grade_A']) # 0.01759

# 信息增益测试样例2
print(compute_information_gains(train_data, one_hot_features, target)['term_ 60 months']) # 0.01429

# 信息增益测试样例3
print(compute_information_gains(train_data, one_hot_features, target)['grade_B']) # 0.00370

0.017591980178887745
0.014291850329400169
0.0037049200345330435


### 4.2 信息增益率
信息增益率：

$$
\mathrm{Gain\_ratio}(D, a) = \frac{\mathrm{Gain}(D, a)}{\mathrm{IV}(a)}
$$

其中

$$
\mathrm{IV}(a) = - \sum^V_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \log_2 \frac{\vert D^v \vert}{\vert D \vert}
$$

完成计算所有特征信息增益率的函数  
**这里要完成四个部分**

In [25]:
def compute_information_gain_ratios(data, features, target, annotate = False):
    '''
    计算所有特征的信息增益率并保存起来
    
    Parameter
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    gain_ratios: dict, key: str, 特征名
                       value: float，信息增益率
    '''
    
    gain_ratios = dict()
    
    # 对所有的特征进行遍历，使用当前的划分方法对每个特征进行计算
    for feature in features:
        
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的信息熵
        left_entropy = information_entropy(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target) / (len(left_split_target) + len(right_split_target))

        # 计算右子树的信息熵
        right_entropy =  information_entropy(right_split_target)                     # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight =  len(right_split_target) / (len(left_split_target) + len(right_split_target))   # YOUR CODE HERE
        
        # 计算当前结点的信息熵
        current_entropy = information_entropy(data[target])
        
        # 计算当前结点的信息增益
        
        gain =  current_entropy - (left_weight * left_entropy + right_weight * right_entropy)                                     # YOUR CODE HERE
        
        # 计算IV公式中，当前特征为0的值
        if left_weight == 0:
            left_IV = 0
        else:
            left_IV = left_weight * np.log2(left_weight)
        
        # 计算IV公式中，当前特征为1的值
        if right_weight == 0:
            right_IV = 0
        else:
            right_IV =  - right_weight * np.log2(right_weight)                             # YOUR CODE HERE
        
        # IV 等于所有子树IV之和的相反数
        IV = - (left_IV + right_IV)
            
        # 计算使用当前特征划分的信息增益率
        # 这里为了防止IV是0，导致除法得到np.inf（无穷），在分母加了一个很小的小数
        gain_ratio = gain / (IV + np.finfo(np.longdouble).eps)
        
        # 信息增益率的存储
        gain_ratios[feature] = gain_ratio
        
        if annotate:
            print(" ", feature, gain_ratio)
            
    return gain_ratios

In [26]:
# 信息增益率测试样例1
print(compute_information_gain_ratios(train_data, one_hot_features, target)['grade_A']) # 0.02573

# 信息增益率测试样例2
print(compute_information_gain_ratios(train_data, one_hot_features, target)['grade_B']) # 0.00417

# 信息增益率测试样例3
print(compute_information_gain_ratios(train_data, one_hot_features, target)['term_ 60 months']) # 0.01970

-0.08364048435688884
-0.02349433005753071
-0.0692273287015029


### 4.3 基尼指数
数据集$D$的基尼值：

$$
\begin{aligned}
\mathrm{Gini}(D) & = \sum^{\vert \mathcal{Y} \vert}_{k=1} \sum_{k' \neq k} p_k p_{k'}\\
& = 1 - \sum^{\vert \mathcal{Y} \vert}_{k=1} p^2_k.
\end{aligned}
$$

属性$a$的基尼指数：

$$
\mathrm{Gini\_index}(D, a) = \sum^V_{v = 1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Gini}(D^v)
$$

完成数据集基尼值的计算  
**这里需要填写三部分**

In [27]:
def gini(labels_in_node):
    '''
    计算一个结点内样本的基尼指数
    
    Paramters
    ----------
    label_in_data: np.ndarray, 样本的标记，如[-1, -1, 1, 1, 1]
    
    Returns
    ---------
    gini: float，基尼指数
    '''
    
    # 统计样本总个数
    num_of_samples = labels_in_node.shape[0]
    
    if num_of_samples == 0:
        return 0
    
    # 统计出1的个数
    num_of_positive = len(labels_in_node[labels_in_node == 1])
    
    # 统计出-1的个数
    num_of_negative =  num_of_samples - num_of_positive                    # YOUR CODE HERE
    
    # 统计正例的概率
    prob_positive = num_of_positive / num_of_samples
    
    # 统计负例的概率
    prob_negative = num_of_negative / num_of_samples                      # YOUR CODE HERE
    
    # 计算基尼值
    gini = 1 - (prob_positive ** 2 + prob_negative ** 2)                               # YOUR CODE HERE
    
    return gini

In [28]:
# 基尼值测试样例1
example_labels = np.array([-1, -1, 1, 1, 1])
print(gini(example_labels)) # 0.48

# 基尼值测试样例2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
print(gini(example_labels)) # 0.40816
    
# 基尼值测试样例3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
print(gini(example_labels)) # 0.40816

# 基尼值测试样例4
example_labels = np.array([-1] * 9 + [1] * 8)
print(gini(example_labels)) # 0.49827

# 基尼值测试样例5
example_labels = np.array([1] * 8)
print(gini(example_labels)) # 0

# 基尼值测试样例6
example_labels = np.array([])
print(gini(example_labels)) # 0

0.48
0.40816326530612246
0.40816326530612246
0.4982698961937716
0.0
0


然后计算所有特征的基尼指数  
**这里需要填写三部分**

In [31]:
def compute_gini_indices(data, features, target, annotate = False):
    '''
    计算使用各个特征进行划分时，各特征的基尼指数
    
    Parameter
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    gini_indices: dict, key: str, 特征名
                       value: float，基尼指数
    '''
    
    gini_indices = dict()
    # 对所有的特征进行遍历，使用当前的划分方法对每个特征进行计算
    for feature in features:
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的基尼值
        left_gini = gini(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target) / (len(left_split_target) + len(right_split_target))

        # 计算右子树的基尼值
        right_gini =  gini(right_split_target)                                 # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight = 1 - left_weight                                # YOUR CODE HERE
        
        # 计算当前结点的基尼指数
        gini_index = left_weight * left_gini + right_weight * right_gini                                  # YOUR CODE HERE
        
        # 存储
        gini_indices[feature] = gini_index
        
        if annotate:
            print(" ", feature, gini_index)
            
    return gini_indices

In [32]:
# 基尼指数测试样例1
print(compute_gini_indices(train_data, one_hot_features, target)['grade_A']) # 0.30095

# 基尼指数测试样例2
print(compute_gini_indices(train_data, one_hot_features, target)['grade_B']) # 0.30568

# 基尼指数测试样例3
print(compute_gini_indices(train_data, one_hot_features, target)['term_ 36 months']) # 0.30055

0.3009520964964362
0.3056855375882364
0.30055418611740065


## 5. 完成最优特征的选择 

到此，我们完成了三种划分策略的实现，接下来就是完成获取最优特征的函数  
**这里需要填写两个部分**

In [33]:
def best_splitting_feature(data, features, target, criterion = 'gini', annotate = False):
    '''
    给定划分方法和数据，找到最优的划分特征
    
    Parameters
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    criterion: str, 使用哪种指标，三种选项: 'information_gain', 'gain_ratio', 'gini'
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    best_feature: str, 最佳的划分特征的名字
    
    '''
    if criterion == 'information_gain':
        if annotate:
            print('using information gain')
        
        # 得到当前所有特征的信息增益
        information_gains = compute_information_gains(data, features, target, annotate)
    
        # information_gains是一个dict类型的对象，我们要找值最大的那个元素的键是谁
        # 根据这些特征和他们的信息增益，找到最佳的划分特征
        best_feature = max(information_gains.items(), key = lambda x: x[1])[0]
        
        return best_feature

    elif criterion == 'gain_ratio':
        if annotate:
            print('using information gain ratio')
        
        # 得到当前所有特征的信息增益率
        gain_ratios = compute_information_gain_ratios(data, features, target, annotate)
    
        # 根据这些特征和他们的信息增益率，找到最佳的划分特征
        best_feature = max(gain_ratios.items(), key=lambda x: x[1])[0]                                                   # YOUR CODE HERE

        return best_feature
    
    elif criterion == 'gini':
        if annotate:
            print('using gini')
        
        # 得到当前所有特征的基尼指数
        gini_indices = compute_gini_indices(data, features, target, annotate)
        
        # 根据这些特征和他们的基尼指数，找到最佳的划分特征
        best_feature = min(gini_indices.items(), key=lambda x: x[1])[0]                                                   # YOUR CODE HERE

        return best_feature
    else:
        raise Exception("传入的criterion不合规!", criterion)

## 6. 判断结点内样本的类别是否为同一类

**这里需要填写两个部分**

In [34]:
def intermediate_node_num_mistakes(labels_in_node):
    '''
    求树的结点中，样本数少的那个类的样本有多少，比如输入是[1, 1, -1, -1, 1]，返回2
    
    Parameter
    ----------
    labels_in_node: np.ndarray, pd.Series
    
    Returns
    ----------
    int：个数
    
    '''
    # 如果传入的array为空，返回0
    if len(labels_in_node) == 0:
        return 0
    
    # 统计1的个数
    num_of_one =  len(labels_in_node[labels_in_node == 1])                                   # YOUR CODE HERE
    
    # 统计-1的个数
    num_of_minus_one = len(labels_in_node[labels_in_node == -1])                              # YOUR CODE HERE
    
    return num_of_one if num_of_minus_one > num_of_one else num_of_minus_one

In [35]:
# 测试样例1
print(intermediate_node_num_mistakes(np.array([1, 1, -1, -1, -1]))) # 2

# 测试样例2
print(intermediate_node_num_mistakes(np.array([]))) # 0

# 测试样例3
print(intermediate_node_num_mistakes(np.array([1]))) # 0

2
0
0


## 7. 创建叶子结点

In [36]:
def create_leaf(target_values):
    '''
    计算出当前叶子结点的标记是什么，并且将叶子结点信息保存在一个dict中
    
    Parameter:
    ----------
    target_values: pd.Series, 当前叶子结点内样本的标记

    Returns:
    ----------
    leaf: dict，表示一个叶结点，
            leaf['splitting_features'], None，叶结点不需要划分特征
            leaf['left'], None，叶结点没有左子树
            leaf['right'], None，叶结点没有右子树
            leaf['is_leaf'], True, 是否是叶子结点
            leaf['prediction'], int, 表示该叶子结点的预测值
    '''
    # 创建叶子结点
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}
   
    # 数结点内-1和+1的个数
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # 叶子结点的标记使用少数服从多数的原则，为样本数多的那类的标记，保存在 leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1
    else:
        leaf['prediction'] = -1

    # 返回叶子结点
    return leaf

## 8. 递归地创建决策树
递归的创建决策树  
递归算法终止的三个条件：
1. 如果结点内所有的样本的标记都相同，该结点就不需要再继续划分，直接做叶子结点即可
2. 如果结点所有的特征都已经在之前使用过了，在当前结点无剩余特征可供划分样本，该结点直接做叶子结点
3. 如果当前结点的深度已经达到了我们限制的树的最大深度，直接做叶子结点

**这里需要填写五个部分**

In [37]:
def decision_tree_create(data, features, target, criterion = 'gini', current_depth = 0, max_depth = 10, annotate = False):
    '''
    Parameter:
    ----------
    data: pd.DataFrame, 数据

    features: iterable, 特征组成的可迭代对象，比如一个list

    target: str, 标记的名字

    criterion: 'str', 特征划分方法，只支持三种：'information_gain', 'gain_ratio', 'gini'

    current_depth: int, 当前深度，递归的时候需要记录

    max_depth: int, 树的最大深度，我们设定的树的最大深度，达到最大深度需要终止递归

    Returns:
    ----------
    dict, dict['is_leaf']          : False, 当前顶点不是叶子结点
          dict['prediction']       : None, 不是叶子结点就没有预测值
          dict['splitting_feature']: splitting_feature, 当前结点是使用哪个特征进行划分的
          dict['left']             : dict
          dict['right']            : dict
    '''
    
    if criterion not in ['information_gain', 'gain_ratio', 'gini']:
        raise Exception("传入的criterion不合规!", criterion)
    
    # 复制一份特征，存储起来，每使用一个特征进行划分，我们就删除一个
    remaining_features = features[:]
    
    # 取出标记值
    target_values = data[target]
    print("-" * 50)
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))

    # 终止条件1
    # 如果当前结点内所有样本同属一类，即这个结点中，各类别样本数最小的那个等于0
    # 使用前面写的intermediate_node_num_mistakes来完成这个判断
    if intermediate_node_num_mistakes(target_values) == 0: 
        print("Stopping condition 1 reached.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 终止条件2
    # 如果已经没有剩余的特征可供分割，即remaining_features为空
    
    if  len(remaining_features) == 0:                                                                # YOUR CODE HERE
        print("Stopping condition 2 reached.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 终止条件3
    # 如果已经到达了我们要求的最大深度，即当前深度达到了最大深度
    
    if  current_depth >= max_depth:                                                                # YOUR CODE HERE
        print("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values)   # 创建叶子结点

    # 找到最优划分特征
    # 使用best_splitting_feature这个函数
    
    splitting_feature = best_splitting_feature(data,features,target,criterion)
    
    # 使用我们找到的最优特征将数据划分成两份
    # 左子树的数据
    left_split = data[data[splitting_feature] == 0]
    
    # 右子树的数据
    right_split = data[data[splitting_feature] == 1]                                                    # YOUR CODE HERE
    
    # 现在已经完成划分，我们要从剩余特征中删除掉当前这个特征
    remaining_features.remove(splitting_feature)
    
    # 打印当前划分使用的特征，打印左子树样本个数，右子树样本个数
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # 如果使用当前的特征，将所有的样本都划分到一棵子树中，那么就直接将这棵子树变成叶子结点
    # 判断左子树是不是“完美”的
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    
    # 判断右子树是不是“完美”的
    if len(right_split) == len(data):
        print("Creating right node.")
        return create_leaf(right_split[target])                                                      # YOUR CODE HERE

    # 递归地创建左子树
    left_tree = decision_tree_create(left_split, remaining_features, target, criterion, current_depth + 1, max_depth, annotate)
    
    # 递归地创建右子树
    
    right_tree =  decision_tree_create(right_split, remaining_features, target, criterion, current_depth + 1, max_depth, annotate)                                                  # YOUR CODE HERE

    # 返回树的非叶子结点
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

训练一个模型

In [38]:
my_decision_tree = decision_tree_create(train_data, one_hot_features, target, 'gini', max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature term_ 36 months. (14831, 58733)
--------------------------------------------------
Subtree, depth = 1 (14831 data points).
Split on feature grade_F. (13003, 1828)
--------------------------------------------------
Subtree, depth = 2 (13003 data points).
Split on feature grade_E. (9818, 3185)
--------------------------------------------------
Subtree, depth = 3 (9818 data points).
Split on feature home_ownership_RENT. (6796, 3022)
--------------------------------------------------
Subtree, depth = 4 (6796 data points).
Split on feature grade_G. (6507, 289)
--------------------------------------------------
Subtree, depth = 5 (6507 data points).
Split on feature grade_D. (4368, 2139)
--------------------------------------------------
Subtree, depth = 6 (4368 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 

Split on feature emp_length_1 year. (9754, 987)
--------------------------------------------------
Subtree, depth = 6 (9754 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (987 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 5 (34 data points).
Split on feature emp_length_10+ years. (27, 7)
--------------------------------------------------
Subtree, depth = 6 (27 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (7 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 4 (9727 data points).
Split on feature emp_length_< 1 year. (9186, 541)
--------------------------------------------------
Subtree, depth = 5 (9186 data points).
Split on feature emp_length_9 years. (8807, 379)
--------------------

现在，模型就训练好了

## 9. 预测

接下来我们需要完成预测函数

In [39]:
def classify(tree, x, annotate = False):
    '''
    递归的进行预测，一次只能预测一个样本
    
    Parameters
    ----------
    tree: dict
    
    x: pd.Series，待预测的样本
    
    annotate： boolean, 是否显示注释
    
    Returns
    ----------
    返回预测的标记
    '''
    if tree['is_leaf']:
        if annotate:
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

我们取测试集第一个样本来测试

In [40]:
test_sample = test_data.iloc[0]
print(test_sample)

safe_loans                     1
grade_A                    False
grade_B                    False
grade_C                    False
grade_D                    False
grade_E                     True
grade_F                    False
grade_G                    False
term_ 36 months             True
term_ 60 months            False
home_ownership_MORTGAGE     True
home_ownership_OTHER       False
home_ownership_OWN         False
home_ownership_RENT        False
emp_length_1 year          False
emp_length_10+ years       False
emp_length_2 years          True
emp_length_3 years         False
emp_length_4 years         False
emp_length_5 years         False
emp_length_6 years         False
emp_length_7 years         False
emp_length_8 years         False
emp_length_9 years         False
emp_length_< 1 year        False
Name: 37225, dtype: object


In [41]:
print('True class: %s ' % (test_sample['safe_loans']))
print('Predicted class: %s ' % classify(my_decision_tree, test_sample))

True class: 1 
Predicted class: 1 


打印出使用决策树判断的过程

In [42]:
classify(my_decision_tree, test_sample, annotate=True)

Split on term_ 36 months = True
Split on grade_A = False
Split on grade_B = False
Split on grade_C = False
Split on home_ownership_MORTGAGE = True
Split on grade_G = False
At leaf, predicting 1


1

## 10. 在测试集上对我们的模型进行评估

In [43]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

先来编写一个批量预测的函数，传入的是整个测试集那样的pd.DataFrame，这个函数返回一个np.ndarray，存储模型的预测结果  

In [44]:
def predict(tree, data):
    '''
    按行遍历data，对每个样本进行预测，将值存在prediction中，最后返回np.ndarray
    
    Parameter
    ----------
    tree: dict, 模型
    
    data: pd.DataFrame, 数据
    
    Returns
    ----------
    predictions：np.ndarray, 模型对这些样本的预测结果
    '''
    predictions = np.zeros(len(data)) # 长度和data一样
    
    for i in range(len(data)):
        predictions[i] = classify(tree,data.iloc[i])
    
    return predictions

## 11. 请你计算使用不同评价指标得到模型的四项指标的值，填写在下方表格内
**树的最大深度为6**  
**这里需要填写一个部分**

In [45]:

my_decision_tree_list = ['gini','information_gain','gain_ratio']


# 创建字典来存储不同划分标准下的四项指标值
evaluation_results = {}

for term in my_decision_tree_list:
    # 使用 term 作为划分标准创建决策树
    my_decision_tree = decision_tree_create(train_data, one_hot_features, target, term, max_depth=6, annotate=False)

    # 预测测试集
    predictions = predict(my_decision_tree, test_data)

    # 计算四项指标
    accuracy = accuracy_score(test_data[target], predictions)
    precision = precision_score(test_data[target], predictions, average='binary')
    recall = recall_score(test_data[target], predictions, average='binary')
    f1 = f1_score(test_data[target], predictions, average='binary')

    # 存储四项指标值
    evaluation_results[term] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-Score': f1}

# 打印结果
for term, metrics in evaluation_results.items():
    print(f"Evaluation results for decision tree using '{term}' criterion:")
    print("Accuracy:", metrics['Accuracy'])
    print("Precision:", metrics['Precision'])
    print("Recall:", metrics['Recall'])
    print("F1-Score:", metrics['F1-Score'])
    print()







--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature term_ 36 months. (14831, 58733)
--------------------------------------------------
Subtree, depth = 1 (14831 data points).
Split on feature grade_F. (13003, 1828)
--------------------------------------------------
Subtree, depth = 2 (13003 data points).
Split on feature grade_E. (9818, 3185)
--------------------------------------------------
Subtree, depth = 3 (9818 data points).
Split on feature home_ownership_RENT. (6796, 3022)
--------------------------------------------------
Subtree, depth = 4 (6796 data points).
Split on feature grade_G. (6507, 289)
--------------------------------------------------
Subtree, depth = 5 (6507 data points).
Split on feature grade_D. (4368, 2139)
--------------------------------------------------
Subtree, depth = 6 (4368 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 

Split on feature emp_length_9 years. (8807, 379)
--------------------------------------------------
Subtree, depth = 6 (8807 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (379 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 5 (541 data points).
Split on feature grade_C. (541, 0)
Creating leaf node.
--------------------------------------------------
Subtree, depth = 2 (13101 data points).
Split on feature home_ownership_MORTGAGE. (5830, 7271)
--------------------------------------------------
Subtree, depth = 3 (5830 data points).
Split on feature emp_length_3 years. (5283, 547)
--------------------------------------------------
Subtree, depth = 4 (5283 data points).
Split on feature emp_length_1 year. (4705, 578)
--------------------------------------------------
Subtree, depth = 5 (4705 data points).
Split on feature emp_length_7 yea

Split on feature home_ownership_MORTGAGE. (10775, 9727)
--------------------------------------------------
Subtree, depth = 4 (10775 data points).
Split on feature emp_length_1 year. (9785, 990)
--------------------------------------------------
Subtree, depth = 5 (9785 data points).
Split on feature home_ownership_OTHER. (9754, 31)
--------------------------------------------------
Subtree, depth = 6 (9754 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (31 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 5 (990 data points).
Split on feature home_ownership_OTHER. (987, 3)
--------------------------------------------------
Subtree, depth = 6 (987 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (3 data points).
Stopping condition 1 reached.
---------------------

Split on feature term_ 60 months. (0, 1166)
Creating right node.
--------------------------------------------------
Subtree, depth = 3 (5025 data points).
Split on feature term_ 60 months. (0, 5025)
Creating right node.
--------------------------------------------------
Subtree, depth = 2 (8636 data points).
Split on feature term_ 60 months. (0, 8636)
Creating right node.
--------------------------------------------------
Subtree, depth = 1 (58733 data points).
Split on feature term_ 60 months. (58733, 0)
Creating leaf node.
Evaluation results for decision tree using 'gini' criterion:
Accuracy: 0.8117366392757376
Precision: 0.8128399100388468
Recall: 0.9980168193799422
F1-Score: 0.8959603357935657

Evaluation results for decision tree using 'information_gain' criterion:
Accuracy: 0.8122056154802928
Precision: 0.8122387390142941
Recall: 0.9999497928956947
F1-Score: 0.8963724740087312

Evaluation results for decision tree using 'gain_ratio' criterion:
Accuracy: 0.8122463960198193
Precisi

树的最大深度为6  

###### 双击此处编写

划分标准|精度|查准率|查全率|F1
-|-|-|-|-
信息增益|0.8122463960198193|0.8122463960198193|1.0|0.8963973086703121
信息增益率|0.8122056154802928|0.8122387390142941|0.9999497928956947|0.8963724740087312
基尼指数|0.8117366392757376|0.8128399100388468|0.9980168193799422|0.8959603357935657